In [45]:
import numpy as np
import torch
from collections import Counter
import re
import pickle as pkl
from sklearn import svm
import pandas as pd

In [46]:
# Function to load word vectors pre-trained on Google News
# Arguments: None
# Returns: w2v (dict)
# Where, w2v (dict) is a dictionary with words as keys (lowercase) and vectors as values
def load_w2v():
    with open('./Data/w2v.pkl', 'rb') as fin:
        return pkl.load(fin)
    
def get_tokens(doc):
	tokens = re.split(r"[^A-Za-z0-9-']", doc)
	tokens = list(filter(len, tokens))
	return tokens


def word2vec_rep(docs):
    '''  DESCRIPTION:   Function to get word2vec averge representations. Input is a collection of documents. Docs are tokenized and for each line the average representation
                        is computed from its constituent tokens. Unknownwords are skipped.
    
         ARGUMENTS: docs: A list of strings, each string represents a document
         
         RETURNS:   mat (numpy.ndarray) of size (len(docs), dim) mat is a two-dimensional numpy array containing vector representation for ith document (in input list docs) in ith row
                    dim represents the dimensions of word vectors, here dim = 300 for Google News pre-trained vectors
    
    '''
    # Declare variables
    freqs = Counter()
    docFreqs = [Counter() for d in docs]
    docTokens = [[] for d in docs]
    voc = []

    # Build Vocabulary
    for i,d in enumerate(docs):
        docTokens = get_tokens(d.lower())                          # get tokens
        #isWord = [t in stopwords for t in tokens]       # see which word is stop word
        #docTokens[i] = [t for (t,i) in zip(tokens,isWord) if not i]# eliminate stopwords from furhter consideration    
        #docTokens[i] = [t for (t,i) in zip(tokens,isWord) if not i]# eliminate stopwords from furhter consideration    
        docFreqs[i] = Counter(docTokens)                   # count the freqs os this docs' tokens 
        freqs += Counter(docTokens)                        # Add the doc's freqs to total freqs


    # Dummy matrix
    dim = 300
    mat = np.zeros((len(docs), dim))
    w2v = load_w2v()

    # Create a sorted vocabulary out of the unique terms. Declare the matrix that hold the per doc
    # bag-of-word represeantions in terms of appeared token freqeuencies!
    voc = list(freqs.keys())
    voc.sort()

    # Build averaged representations
    cnt = 0
    embedding = np.zeros(dim)
    for i in range(len(docs)):
        for j, v in enumerate(voc):
            if v in w2v:
                embedding += (w2v[v] * docFreqs[i][v])
                cnt += docFreqs[i][v]
            # else: # word not in w2v, just consider it as adding 0's
                # cnt += 1
        cnt = cnt if cnt > 0 else 1
        mat[i] = embedding / cnt # average the summed embeddings
        cnt = 0
        embedding.fill(0) 


    return mat

tweetFile = './Data/training-Obama-Romney-tweets_corrected2_normalized_no_stop_words.txt'

# Read all lines of tweet file, store them as list of strings
file1 = open(tweetFile, 'r') 
lines = file1.readlines()

# Get the average word 2 vec representation of all tweets. Unknown words are omitted.
m = word2vec_rep(lines)

# Save reps to disk for future use
saveFile = './Data/avg_w2v_rep.npy'
np.save(saveFile,m)

## Get labels

In [51]:
labelFile = './Data/training-Obama-Romney-tweets_corrected2_normalized_no_stop_words_labels.txt'
labels = np.loadtxt(labelFile)
print(m.shape, labels.shape)

(5624, 300) (5624,)


## SVM Sentiment CLassification
### Train Classifier

In [56]:
n = 0.8  # for 2 random indices
dataSize = m.shape[0]
trainSize = int(n * dataSize)
trainIdxs = np.random.choice(dataSize, trainSize, replace=False)  
print(trainIdxs)

[1376 4711 2137 ... 5419 4648 3384]


In [52]:
# One vs one case
#clf = svm.SVC()
#clf.fit(m, labels)
# One-vs rest
lin_clf = svm.LinearSVC()
lin_clf.fit(m, labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [53]:
dec = lin_clf.decision_function([[1]])
dec.shape[1]

ValueError: X has 1 features per sample; expecting 300